In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

import string
import datetime
import time
import csv
import ast
import import_ipynb
import NBAInjuryAnalysis
import NBAPlayerAnalysis
import NBANetRatingAnalysis

importing Jupyter notebook from NBAInjuryAnalysis.ipynb
importing Jupyter notebook from NBAPlayerAnalysis.ipynb
importing Jupyter notebook from NBANetRatingAnalysis.ipynb


In [2]:
# global variables
%store -r playersToIds
%store -r idsToPlayers
%store -r datesDict
%store -r playoffDates
%store -r statsNBAMap
%store -r apd
%store -r allNames

In [3]:
def collectPlayerData(playerName):
    playerData = {}
    playerData.update(NBAInjuryAnalysis.collectInjuryData(playerName))
    print('one')
    playerData.update(NBANetRatingAnalysis.collectNetRatings(playerName))
    print('two')
    playerData.update(NBAPlayerAnalysis.collectPlayerHistory(playerName))
    print('three')
    return playerData

def findPlayers(year, allPlayerData, start, end):
    url = "https://www.basketball-reference.com/leagues/NBA_" + year + "_totals.html"
    html = urlopen(url)
    soup = BeautifulSoup(html, features="lxml")
    players = soup.find('tbody').findAll('tr')
    print(len(players))
    for i in range(start, end):
        player = players[i]
        if player['class'] == ["thead"]:
            continue
        playerName = player.find('a').text
        if playerName not in allPlayerData and playerName not in allNames:
            print(playerName)
            try:
                playerData = collectPlayerData(playerName)
                allPlayerData[playerName] = playerData
            except KeyboardInterrupt:
                print('keyboard interrupt')
                return allPlayerData
            except Exception as e:
                print("error:", playerName)
                print(e)
        
    return allPlayerData

def writeData(allPlayerData, num):
    w = csv.writer(open(str(num) + "output.csv", "a"))
    for key, val in allPlayerData.items():
        w.writerow([key, val])
        
def readData(num):
    with open(str(num) + 'output.csv', mode='r') as infile:
        reader = csv.reader(infile)
        with open(str(num) + 'reader.csv', mode='w') as outfile:
            writer = csv.writer(outfile)
            mydict = {rows[0]:rows[1] for rows in reader}

    for name in mydict:
        val = mydict[name]
        mydict[name] = eval(val)

    return mydict

In [4]:
# collectPlayerData("Michael Jordan")